# [Grounding with Bing Search](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview)
![image.png](attachment:fd7610d6-cec1-40f6-a7bd-695163cdd9f0.png)<br/><br/>
**Grounding with Bing Search** allows our Azure AI Agents to incorporate real-time public web data when generating responses.<br/>
We need to create a Grounding with Bing Search resource, and then connect this resource to your Azure AI Agents.
<br/>When a user sends a query, Azure AI Agents decide if Grounding with Bing Search should be leveraged or not. If so, it will leverage Bing to search over public web data and return relevant chunks. Lastly, Azure AI Agents will use returned chunks to generate a response.<br/><br/>
![image.png](attachment:4d2c4215-ec52-4749-ba2c-dafd496f7d44.png)
![image.png](attachment:6785d8e2-0046-4e34-b817-5a351a8209a6.png)
![image.png](attachment:1324e024-504a-47b1-b6d3-ce2982927f28.png)
![image.png](attachment:e187d760-5db3-4063-b398-26782d439672.png)


## Bing Search Setup

### 1 Creating the Grounding with Bing Search resource  
- Create a **Grounding with Bing Search** resource within the Resource Group. This resource allows the Foundry project to perform web requests to enrich answers with up-to-date content.

### 8.2 Connecting the Resource to AI Foundry  
- Go to your AI Foundry project.  
- In the **Overview** section click on open in **Management Center**.  
- In the **Connected resources** section, click on **New connection > Grounding with Bing Search**.  
- Select your service and add the connection.  
- Use **API key** for authentication.  
- In the `credentials.env` file, save the name of the newly created connection in the variable **BING_CONNECTION_NAME**.


# Constants

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv(dotenv_path='../infra/credentials.env')
model_name = os.environ["MODEL_DEPLOYMENT_NAME"]
project_connection_string = os.environ["AI_PROJECT_CONNECTION_STRING"]

print(f'Project Connection String: <...{project_connection_string[-30:]}>')

Project Connection String: <...ing-rg;aifoundry-upskilling-pj>


# Create AI Foundry Project Client

In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool # <<<<<<<<<<<<<<< SPECIFIC FOR BING SEARCH
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

project_client.scope

{'subscription_id': 'dbc342d5-96b5-4aef-a49d-5f6cbd7db6ce',
 'resource_group_name': 'aifoundry-upskilling-rg',
 'project_name': 'aifoundry-upskilling-pj'}

# Create the BingGroundingTool in 2 steps

## First, retrieve the BING Connection already associated to the AI Foundry project...

In [3]:
bing_connection = project_client.connections.get(connection_name=os.environ["BING_CONNECTION_NAME"])

print(bing_connection)

{
 "name": "bingsearchaievaluation",
 "id": "/subscriptions/dbc342d5-96b5-4aef-a49d-5f6cbd7db6ce/resourceGroups/aifoundry-upskilling-rg/providers/Microsoft.MachineLearningServices/workspaces/aifoundry-upskilling-pj/connections/bingsearchaievaluation",
 "authentication_type": "ApiKey",
 "connection_type": "ApiKey",
 "endpoint_url": "https://api.bing.microsoft.com",
 "key": null
 "token_credential": null
}



## ...then, initialize `BingGroundingTool` and add the connection id to it

In [4]:
bing = BingGroundingTool(connection_id=bing_connection.id)
print(f"bing.definitions: {bing.definitions}")
print(f"bing.resources: {bing.resources}")

bing.definitions: [{'type': 'bing_grounding', 'bing_grounding': {'connections': [{'connection_id': '/subscriptions/dbc342d5-96b5-4aef-a49d-5f6cbd7db6ce/resourceGroups/aifoundry-upskilling-rg/providers/Microsoft.MachineLearningServices/workspaces/aifoundry-upskilling-pj/connections/bingsearchaievaluation'}]}}]
bing.resources: {}


# ***Load*** or ***Create*** an AI Foundry Agent

In [5]:
project_client.agents.list_agents()['data']

[]

In [6]:
# Agent creation
# Notices that FileSearchTool as tool and tool_resources must be added or the assistant unable to search the file

assistant_id = "" # ex: asst_j1qWBdGsjbK4hHcO0M0n3M5p

if assistant_id != "":
    agent = project_client.agents.get_agent(assistant_id=assistant_id)
else:
    agent = project_client.agents.create_agent(
        model=model_name,
        name="aiagent-PYTHON-bing",
        instructions="You are an helpful assistant. You search the web and summarize the results clearly and concisely",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )

print(f"Agent: {agent}")

Agent: {'id': 'asst_2r2N3IhcwLUvY3k05irBOyrd', 'object': 'assistant', 'created_at': 1744704484, 'name': 'aiagent-PYTHON-bing', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are an helpful assistant. You search the web and summarize the results clearly and concisely', 'tools': [{'type': 'bing_grounding', 'bing_grounding': {'connections': [{'connection_id': '/subscriptions/dbc342d5-96b5-4aef-a49d-5f6cbd7db6ce/resourceGroups/aifoundry-upskilling-rg/providers/Microsoft.MachineLearningServices/workspaces/aifoundry-upskilling-pj/connections/bingsearchaievaluation'}]}}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}


# Create the thread and attach a new message to it

In [7]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}\n")

Created thread: {'id': 'thread_13sy1e9PyE3PBJiMGPGJGysm', 'object': 'thread', 'created_at': 1744704485, 'metadata': {}, 'tool_resources': {}}



In [8]:
# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id, 
    role="user", 
    content="What is the top news today?", # "What is the top news today", "Quali sono i programmi TV stasera?"
)
print(f"Created message: {message}")

Created message: {'id': 'msg_2cbLlR24MnkVWrJ6syrWpOgA', 'object': 'thread.message', 'created_at': 1744704485, 'assistant_id': None, 'thread_id': 'thread_13sy1e9PyE3PBJiMGPGJGysm', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What is the top news today?', 'annotations': []}}], 'attachments': [], 'metadata': {}}


In [9]:
project_client.agents.list_messages(thread_id=thread.id).as_dict()

{'object': 'list',
 'data': [{'id': 'msg_2cbLlR24MnkVWrJ6syrWpOgA',
   'object': 'thread.message',
   'created_at': 1744704485,
   'assistant_id': None,
   'thread_id': 'thread_13sy1e9PyE3PBJiMGPGJGysm',
   'run_id': None,
   'role': 'user',
   'content': [{'type': 'text',
     'text': {'value': 'What is the top news today?', 'annotations': []}}],
   'attachments': [],
   'metadata': {}}],
 'first_id': 'msg_2cbLlR24MnkVWrJ6syrWpOgA',
 'last_id': 'msg_2cbLlR24MnkVWrJ6syrWpOgA',
 'has_more': False}

# Run the agent synchronously

In [10]:
%%time
# Create and process assistant run in thread with tools
run = project_client.agents.create_and_process_run\
    (thread_id=thread.id, agent_id=agent.id, tool_choice='required',)

print(f"Run finished with status: {run.status}.\n\nRun: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: completed.

Run: {'id': 'run_6ycCAXyGUqaCVUSqoXoass9K', 'object': 'thread.run', 'created_at': 1744704487, 'assistant_id': 'asst_2r2N3IhcwLUvY3k05irBOyrd', 'thread_id': 'thread_13sy1e9PyE3PBJiMGPGJGysm', 'status': 'completed', 'started_at': 1744704487, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1744704492, 'required_action': None, 'last_error': None, 'model': 'gpt-4o', 'instructions': 'You are an helpful assistant. You search the web and summarize the results clearly and concisely', 'tools': [{'type': 'bing_grounding', 'bing_grounding': {'connections': [{'connection_id': '/subscriptions/dbc342d5-96b5-4aef-a49d-5f6cbd7db6ce/resourceGroups/aifoundry-upskilling-rg/providers/Microsoft.MachineLearningServices/workspaces/aifoundry-upskilling-pj/connections/bingsearchaievaluation'}]}}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy

# Fetch messages from the thread after the agent run execution

In [11]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages.data), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}\n")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages:


===== MESSAGE 1 =====

CONTENT 1 (MessageTextContent) --> Text: What is the top news today?


===== MESSAGE 2 =====

CONTENT 1 (MessageTextContent) --> Text: Here are some of the top news stories for April 15, 2025:

1. Ukraine can potentially join the EU, but not NATO, according to Russia during peace talks to end the ongoing war【3:2†source】.
2. The Philippines has raised concerns regarding the South China Sea during ASEAN-China talks【3:9†source】.
3. Apple shares have increased following relief from tariffs【3:9†source】.

These headlines capture important political and economic developments occurring today.

>>> Annotation in MessageTextContent 1 of message 2: 【3:2†source】

>>> Annotation in MessageTextContent 1 of message 2: 【3:9†source】

>>> Annotation in MessageTextContent 1 of message 2: 【3:9†source】



In [12]:
from azure.ai.projects.models import FilePurpose, MessageRole

# Get the last message from the sender
last_msg = messages.get_last_text_message_by_role(MessageRole.AGENT)
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

Last Message: Here are some of the top news stories for April 15, 2025:

1. Ukraine can potentially join the EU, but not NATO, according to Russia during peace talks to end the ongoing war【3:2†source】.
2. The Philippines has raised concerns regarding the South China Sea during ASEAN-China talks【3:9†source】.
3. Apple shares have increased following relief from tariffs【3:9†source】.

These headlines capture important political and economic developments occurring today.


# Print annotations from the messages

In [13]:
print(f"Number of annotation(s): {len(last_msg.text.annotations)}")

for annotation in last_msg.text.annotations:
    print(annotation["text"], annotation["url_citation"]["url"])

Number of annotation(s): 3
【3:2†source】 https://www.ndtv.com/video/top-headlines-of-the-day-april-15-2025-926594
【3:9†source】 https://www.schooldekho.org/school/blog/details/school-assembly-news-headlines-today-15-april-2025-1781
【3:9†source】 https://www.schooldekho.org/school/blog/details/school-assembly-news-headlines-today-15-april-2025-1781


# Run Steps

In [14]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)

print(f"Nr of run step(s): {len(run_steps['data'])}\n")
i=0
for rs in run_steps["data"]:
    i += 1
    print(f"Run step {i}: {rs}", '\n')

Nr of run step(s): 2

Run step 1: {'id': 'step_prwksvL20SxLVlAp7KIxFf2f', 'object': 'thread.run.step', 'created_at': 1744704489, 'run_id': 'run_6ycCAXyGUqaCVUSqoXoass9K', 'assistant_id': 'asst_2r2N3IhcwLUvY3k05irBOyrd', 'thread_id': 'thread_13sy1e9PyE3PBJiMGPGJGysm', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1744704492, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_M3NwV0TO9tUMbOwvHKr1hcBv'}}, 'usage': {'prompt_tokens': 1237, 'completion_tokens': 104, 'total_tokens': 1341, 'prompt_token_details': {'cached_tokens': 0}}} 

Run step 2: {'id': 'step_CZWIbv9GFWJYcivQhZjBmoK9', 'object': 'thread.run.step', 'created_at': 1744704488, 'run_id': 'run_6ycCAXyGUqaCVUSqoXoass9K', 'assistant_id': 'asst_2r2N3IhcwLUvY3k05irBOyrd', 'thread_id': 'thread_13sy1e9PyE3PBJiMGPGJGysm', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at':

# START Teardown

In [ ]:
# Delete all agents

print(f"{len(project_client.agents.list_agents()['data'])} agent(s) will now be deleted")

i=0
for pca in project_client.agents.list_agents()['data']:
    i += 1
    project_client.agents.delete_agent(pca.id)
    print(f"\n{i} - Agent {pca.name} has been deleted")